### In this notebook we are going to Predict Tesla Stock prices by using LSTM.

# Contents
## * Setup
## * Exploring Dataset
## * Data Preprocessing
## * Building LSTM model
## * Actual vs Predicted Prices

# Setup

In [ ]:
#Importing Liabraries
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime
from sklearn.preprocessing import MinMaxScaler

#Importing Keras libraries and packages
from keras.models import Model,Sequential
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, add, concatenate
from keras.preprocessing import text, sequence
from keras.layers import LSTM,Dropout
plt.style.use('fivethirtyeight')

In [ ]:
#loading Dataset
df = pd.read_csv('../input/tesla-stocks-history/TSLA.csv',index_col="Date",parse_dates=True)
df

# Exploring Dataset

In [ ]:
#Number of Rows and Columns
df.shape

In [ ]:
df.isna().any()

In [ ]:
df.info()

In [ ]:
#Visualize the Closing Price history
plt.figure(figsize=(16,8))
plt.title('Close Price History')
plt.plot(df['Close'])
plt.xlabel('Date',fontsize=18)
plt.ylabel('Close Price USD ($)', fontsize=18)
plt.show()

# Data Preprocessing 

In [ ]:
#Create a new dataframe with only Close Column
data = df.filter(['Close'])

#Convert dataframe to a numpy array
dataset = data.values

#Get the number of rows to train the model
training_data_len = math.ceil( len(dataset) * .8) # 80% data for training
training_data_len

In [ ]:
#Scale the data
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)

scaled_data

In [ ]:
#Create the training dataset
train_data = scaled_data[0:training_data_len , :]
#Split the data into x_train and y_train datasets
x_train = []
y_train = []

for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])
    if i<=61:
        print(x_train)
        print(y_train)
        print()


In [ ]:
#Convert the x_train and y_train to numpy arrays
x_train,y_train = np.array(x_train), np.array(y_train)

x_train.shape

In [ ]:
#Reshape the data
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1],1))
x_train.shape

# 3. Building LSTM model

In [ ]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape = (x_train.shape[1], 1)))
model.add(LSTM(50, return_sequences = False))
model.add(Dense(25))
model.add(Dense(1))

#Compiling the model
model.compile(optimizer='adam', loss='mean_squared_error')

#Train the model
model.fit(x_train, y_train, batch_size=1, epochs=1)

In [ ]:
#Create the Testing dateset
test_data = scaled_data[training_data_len - 60: , :]

#Create the datasets x_test and y_test
x_test = []
y_test = dataset[training_data_len:, :]
for i in range(60, len(test_data)):
    x_test.append(test_data[i-60:i, 0])
    
#Converting x-test to numpy array
x_test = np.array(x_test)

#Reshape the data
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1 ))

#Get the models predicted price values
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

# Actual vs Predicted Prices

In [ ]:
#Plot the data
train = data[:training_data_len]
valid = data[training_data_len:]
valid['Predictions'] = predictions

#Visualize the data
plt.figure(figsize=(16,8))
plt.title('Model')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price USD ($)',fontsize=18)
plt.plot(train['Close'])
plt.plot(valid[['Close','Predictions']])
plt.legend(['Train','Val','Predictions'],loc='lower right')
plt.show()

In [ ]:
#Compare valid and Predicted Prices
valid.head()

## Above shows that Predicted values are very close to original values.

In [ ]:
# If you like this notebook, don't forget to upvote .